## Walkthrough of Data Science - Traveler

### * Goal: Predict the country that users will make their first booking in, based on some basic user profile data.

#### * Training data, set of users with correct category (i.e. what country they made their first booking in).

#### * Build a model to accurately predict the country of first booking.

#### * Test data, set of users without the knowledge of outcome.

#### * Task in hand: Preprocessing, how?

## Walkthrough process

#### [1] Pre-processing: Assessing and analyzing data, cleaning, transforming and adding new features
#### [2] Learning model: Constructing and testing learning model
#### [3] Post-processing: Creating final predictions


### LAB 1 CODE - DATA PREPROCESSING (Data Cleaning and Data Transformation)

#### Data Cleaning

In [ ]:
##Exploring Traveler data
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline 

print("Reading data...")
train_file = "./traveler_dataset/train_users_2.csv"
df_train = pd.read_csv(train_file, header = 0,index_col=None)

test_file = "./traveler_dataset/test_users.csv"
df_test = pd.read_csv(test_file, header = 0,index_col=None)

# Combining into one dataset for cleaning
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True, sort=False)
print("Reading data...completed")

# Fixing date formats in Pandas - to_datetime
## Change dates to specific format
print("Fixing timestamps...")
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')
print("Fixing timestamps...completed")

## Removing date_first_booking column
df_all.drop('date_first_booking', axis = 1, inplace = True)
print("Droped date_first_booking column...")

import numpy as np

## Remove outliers function - [1]
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df

## Fixing age column - [2]
print("Fixing age column...")
df_all = remove_outliers(df = df_all, column = 'age', min_val = 15, max_val = 90)
df_all['age'].fillna(-1, inplace = True)
print("Fixing age column...completed")

# Other column missing value - Fill first_affiliate_tracked column
print("Filling first_affiliate_tracked column...")
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)
print("Filling first_affiliate_tracked column...completed")

df_all.head()

#### Data Transformation and Feature Extraction

In [ ]:
# Own implementation of One Hot Encoding - Data Transformation
def convert_to_binary(df, column_to_convert):
    categories = list(df[column_to_convert].drop_duplicates())

    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert[:5] + '_' + cat_name[:10]
        df[col_name] = 0
        df.loc[(df[column_to_convert] == category), col_name] = 1

    return df

# One Hot Encoding
print("One Hot Encoding categorical data...")
columns_to_convert = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

for column in columns_to_convert:
    df_all = convert_to_binary(df=df_all, column_to_convert=column)
    df_all.drop(column, axis=1, inplace=True)
print("One Hot Encoding categorical data...completed")

# Add new date related fields - Creating New Features
print("Adding new fields...")
df_all['day_account_created'] = df_all['date_account_created'].dt.weekday
df_all['month_account_created'] = df_all['date_account_created'].dt.month
df_all['quarter_account_created'] = df_all['date_account_created'].dt.quarter
df_all['year_account_created'] = df_all['date_account_created'].dt.year
df_all['hour_first_active'] = df_all['timestamp_first_active'].dt.hour
df_all['day_first_active'] = df_all['timestamp_first_active'].dt.weekday
df_all['month_first_active'] = df_all['timestamp_first_active'].dt.month
df_all['quarter_first_active'] = df_all['timestamp_first_active'].dt.quarter
df_all['year_first_active'] = df_all['timestamp_first_active'].dt.year
df_all['created_less_active'] = (df_all['date_account_created'] - df_all['timestamp_first_active']).dt.days
print("Adding new fields...completed")


# Drop unnecessary columns
print("Droping fields...")
columns_to_drop = ['date_account_created', 'timestamp_first_active', 'date_first_booking', 'country_destination']
for column in columns_to_drop:
    if column in df_all.columns:
        df_all.drop(column, axis=1, inplace=True)
print("Droping fields...completed")

### What Next? - Data Integration 
- Adding new data **sessions.csv** to **df_all** dataframe


### Milestone: Understanding the sessions.csv data.

In [ ]:
df_all.head()

In [ ]:
## Loading sessions.csv data
print("Reading sessions data...")
sessions_file = "./traveler_dataset/sessions.csv"
df_sessions = pd.read_csv(sessions_file, header = 0,index_col=False)
print("Reading sessions data...completed")
df_sessions.head()

In [ ]:
df_sessions.shape[0]


### Summary:

   - While combining sessions data with the training data, we need to aggregate the sessions data so that there is one row per user (as opposed to many rows for each user)
   
   - Two attributes are striking: **device_type** and **secs_elapsed**
      - These attributes provide important information that could help to more accurately predict which country a user will make a first booking in.
      
   - For example, assume that people spending relatively little time to make a booking on a phone are likely to be making bookings in locations closer to home (i.e. US) than someone spending more time to make a booking on a desktop computer. 
   
   - Remember this is just a theory that needs to be proven, but it is a good reason to ensure it helps in final training dataset.
    

### Milestone: Cleaning and Transforming the Data

- **Goal:** We need to get the final data into a format that can be merged with the data created for **df_all** (i.e., training data + test data)


#### [1] Extract the primary and secondary devices for each user

- How do we determine what a user’s primary and secondary devices are?

- **Solution:** Look at how much time user's spent on each device

<img src="./images/primary_secondary_device_type.jpg" height="500" width="600"/>

- **Note:** By aggregating the data this way, we are also implicitly removing the missing values. (**Hint:** ref. Pandas Groupby) 


#### Determine primary device

In [ ]:

print("Determing primary device...")

In [ ]:
# Selection of all rows for 'user_id', 'device_type', 'secs_elapsed' using .loc operation
sessions_device = df_sessions.loc[:, ['user_id', 'device_type', 'secs_elapsed']]
sessions_device.head(10)

In [ ]:
# Grouping based on 'user_id', 'device_type' the sum of 'secs_elapsed' 
aggregated_lvl1 = sessions_device.groupby(['user_id', 'device_type'], as_index=False, sort=False).aggregate(np.sum)
aggregated_lvl1.head(10)

In [ ]:
# Obtaining the index which is true or false based on first largest device type used by users as matching condition
idx = aggregated_lvl1.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == aggregated_lvl1['secs_elapsed']
idx.head(10)

In [ ]:
# Obtaining the rows of first largest device type used by users based on index
df_sessions_primary = pd.DataFrame(aggregated_lvl1.loc[idx, ['user_id', 'device_type', 'secs_elapsed']])
df_sessions_primary.head(10)

In [ ]:
# Rename the attributes and modify in the df_sessions_primary dataframe
df_sessions_primary.rename(columns = {'device_type':'primary_device','secs_elapsed':'primary_secs'}, inplace=True)
df_sessions_primary.head(10)

#### Transformation: 
- Now convert the primary_device attribute (**df_sessions_primary['primary_device']**) from categorical to numeric form

In [ ]:
# Call user defined One Hot Encoding function
df_sessions_primary = convert_to_binary(df=df_sessions_primary, column_to_convert='primary_device')
df_sessions_primary.head(10)

In [ ]:
# drop the 'primary_device' attribute after one-hot encoding
df_sessions_primary.drop('primary_device', axis=1, inplace=True)
df_sessions_primary.head(10)
print("Determing primary device...completed")

In [ ]:
## Checking ...
df_sessions_primary.head()

#### Exercise 1: Determine Secondary device

In [ ]:

print("Determing secondary device...")

In [ ]:
# Initially drop the primary device index before selecting the remaining device (eg: secondary device)




In [ ]:
### Start code
# Obtaining the index which is true or false based on second largest device type used by users as matching condition




In [ ]:
# Obtaining the rows of second largest device type used by users based on index




In [ ]:
# Rename the attributes and modify in the df_sessions_secondary dataframe




#### End code

#### Exercise 2: Transformation 
- Now convert the secondary_device attribute from categorical into numeric form

In [ ]:
### Start code

In [ ]:
# Call user defined One Hot Encoding function




In [ ]:
# drop the 'secondary_device' attribute after one-hot encoding



print("Determing secondary device...completed")

In [ ]:
## Checking...
df_sessions_secondary.head()
### End code

In [ ]:
df_sessions.head()

#### Exercise 3: 
- Display number of rows and columns in 
   - df_sessions_primary
   - df_sessions_secondary
   - df_sessions

In [ ]:
### Start code




### End code

#### [2] Determine Counts of Actions
- What? Counts of how many times each action was taken by each user

- **Solution:** Two-step process

- **Step1:** Determine the count of each action for each user

<img src="./images/Count_of_actions_Step1.jpg" height="500" width="600"/>

- **Step2:** pivot table

<img src="./images/Count_of_actions_Step2.jpg" height="500" width="600"/>


#### [3] Looping Through the action attributes

- **Problem:** Transformation works for one action attribute at a time, but in the data we have three action attributes: **action, action_type** and **action_detail**


- **Solution:** Repeat **determine counts of actions ([2])** steps for each attribute individually, effectively by creating three separate tables.


- **Effective Approach:** Since this now creates tables where each row represents one user, *JOIN* (SQL concept) these three tables together on the basis of the **user-id**.

In [ ]:
df_sessions.head()

#### Implementation of step 1 and 2

In [ ]:
# Count occurrences of value in a column
def convert_to_counts(df, id_col, column_to_convert):
    #id_list = df[id_col].drop_duplicates()
    #print (id_list.head())
    #Step1
    df_counts = df.loc[:,[id_col, column_to_convert]]  
    df_counts['count'] = 1
    df_counts = df_counts.groupby(by=[id_col, column_to_convert], as_index=False, sort=False).sum()
    print('Step1')
    print (df_counts.head())
    #Step2
    new_df = df_counts.pivot(index=id_col, columns=column_to_convert, values='count') 
    new_df = new_df.fillna(0)
    print ('Step2')
    print (new_df.head())

    # Rename Columns
    categories = list(df[column_to_convert].drop_duplicates())
    print('categories')
    print (categories)
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert + '_' + cat_name
        new_df.rename(columns = {category:col_name}, inplace=True)
        
    return new_df



In [ ]:
# Aggregate and combine actions taken columns
print("Aggregating actions taken...")
session_actions = df_sessions.loc[:,['user_id', 'action', 'action_type', 'action_detail']]
#session_actions = df_sessions.loc[:,['user_id', 'action']]
session_actions.head()

In [ ]:
#columns_to_convert = ['action']
columns_to_convert = ['action', 'action_type', 'action_detail']
session_actions = session_actions.fillna('not provided')
session_actions.head()

In [ ]:
first = True
for column in columns_to_convert:
    print("Converting " + column + " attribute...")
    current_data = convert_to_counts(df=session_actions, id_col='user_id', column_to_convert=column)
    print("Converting " + column + " attribute... finished")

    # If first loop, current data becomes existing data, otherwise merge existing and current
    if first:
        first = False
        actions_data = current_data
    else:
        actions_data = pd.concat([actions_data, current_data], axis=1, join='inner')

In [ ]:
##Checking...
actions_data.head()

#### [4] Finally, combine datasets (Data Integration)

- [4.1] First, combine the two device dataframes (**df_primary** and **df_secondary**) to create a **device** dataframe. 

- [4.2] Then, combine the **device** dataframe with the **actions** dataframe to create a **sessions** dataframe with all the features extracted from *sessions.csv*

- [4.3] Finally, combine the **sessions** dataframe with the **users**(train + test) dataframe computed earlier

In [ ]:
# [4.1] Merge device datasets
print("Combining df_primary and df_secondary to device dataframe...")
df_sessions_primary.set_index('user_id', inplace=True)
df_sessions_secondary.set_index('user_id', inplace=True)
device_data = pd.concat([df_sessions_primary, df_sessions_secondary], axis=1, join="outer", sort=False)
print("Combining df_primary and df_secondary to device dataframe...finished")


In [ ]:
device_data.head()

#### A Note on Outer Join

- [4.1*] Requires outer join because not all users have a secondary device. Doing an outer join here ensures that the dataset includes all users regardless of this fact.

In [ ]:
# [4.2] Merge device and actions datasets
print("Combining device and actions to sessions dataframe...")
combined_results = pd.concat([device_data, actions_data], axis=1, join='outer', sort=False)
df_sessions_complete = combined_results.fillna(0)
print("Combining device and actions to sessions dataframe...finished")


In [ ]:
df_sessions_complete.head()

#### A Note on Outer Join

- [4.2*] Requires outer join just to ensure that if a user is missing from one of the datasets (for whatever reason), we will still capture them.

- Fill any missing values with 0's to ensure not to have any NULL values that may have been generated by these outer joins.

In [ ]:
# [4.3] Merge user and session datasets
print("Combining sessions and users to get final dataframe...")
df_all.set_index('id', inplace=True)
df_all = pd.concat([df_all, df_sessions_complete], axis=1, join='inner', sort = False)
print("Combining sessions and users to get final dataframe...finished")

In [ ]:
df_all.head()

#### A Note on Inner Join

- [4.3*] Requires inner join i.e., want to have final training dataset to only include users that also have sessions data (filters others).


In [ ]:
df_all.columns

#### Exercise 4:
 - Perform data pre-processing on your CP dataset